In [2]:
!pip install datasets transformers rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.2 MB/s 
     |████████████████████████████████| 5.8 MB 65.5 MB/s 
     |████████████████████████████████| 132 kB 64.7 MB/s 
     |████████████████████████████████| 182 kB 79.4 MB/s 
     |████████████████████████████████| 212 kB 79.6 MB/s 
     |████████████████████████████████| 127 kB 80.1 MB/s 
     |████████████████████████████████| 7.6 MB 70.6 MB/s 
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=c7854f8682fa3bc39aa96d80e7d9dd31a6f1dc73bf778c177fe2e533a075fe2a
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge-score
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [4]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [5]:
import transformers


print(transformers.__version__)

4.25.1


In [6]:
model_checkpoint = 't5-small'

In [36]:
from datasets import load_dataset, load_metric 

# scientific_dataset: full text and abstract of papers from arxiv or pubmed
# raw_dataset = load_dataset('scientific_papers', 'arxiv')
# loading in XSum dataset: extreme, 1 sentence summaries of BBC news articles. 
raw_dataset = load_dataset('xsum')
metric = load_metric('rouge')

  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [11]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

In [12]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [14]:
show_random_elements(raw_dataset["train"], num_examples=1)

,document,summary,id
0,"Their car collided with a lorry in the north-eastern city of Polokwane.\nMr Chabane, 54, was minister for public service, a member of the governing ANC's national executive committee and a close associate of President Jacob Zuma.\nHe was in charge of planning Nelson Mandela's funeral.\nMr Zuma said he was ""deeply shocked and saddened"" by his friend's untimely death which was a ""huge loss to government and the country as a whole"".\nHe extended ""heartfelt condolences"" to Mr Chabane's wife Mavis and the families of Sgt Lesiba Sekele and Sgt Lawrence Lentsoane, who also died.\nThe ANC said ""the falling of a great baobab tree"" would leave ""a huge void in the ANC and government"".\nA lorry driver is being investigated over possible manslaughter, reported AFP news agency.\nAs well as alcohol and excessive driving speeds, lack of railway freight is said to be partly responsible for South Africa's high road accident toll - with private motorists forced to share the road with long lines of large mining lorries.","A South African cabinet minister, Collins Chabane, and two of his security staff have been killed in a car crash.",31895879


In [24]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [28]:
# prediction from an LLM is compared with prediction (generated summary) and label (summary from dataset)
generated_summary = ["The Industrial Revolution has been a disaster for humans."]
true_summary = ["The Industrial Revolution and its consequences have been a disaster for the human race."]
results = metric.compute(predictions=generated_summary, references=true_summary)

for key in results.keys():
  print('fmeasure for ' + key + ' : ' + str(results[key].mid.fmeasure))

fmeasure for rouge1 : 0.6086956521739131
fmeasure for rouge2 : 0.4761904761904762
fmeasure for rougeL : 0.6086956521739131
fmeasure for rougeLsum : 0.6086956521739131


In [29]:
from transformers import AutoTokenizer

# tokenizer associated with the picked model     
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [31]:
tokenizer(['hello how are you ?', 'hello how'])

{'input_ids': [[21820, 149, 33, 25, 3, 58, 1], [21820, 149, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1]]}

In [32]:
# t5 models need a prefix of 'summarize: ' or 'translate: ' in the input 

if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [37]:
# inputs and summaries would need to be padded according to the model chosen 
max_input_len = 1024
max_summary_len = 128

def preprocess_function(examples):
    # prepend prefix to the document 
    inputs = [prefix + doc for doc in examples["document"]]
    # tokenize input with max len. would need padding if short or truncation if longer than max len.
    model_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_summary_len, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [40]:
preprocess_function(raw_dataset['train'][:2])

{'input_ids': [[21603, 10, 37, 423, 583, 13, 1783, 16, 20126, 16496, 6, 80, 13, 8, 844, 6025, 4161, 6, 19, 341, 271, 14841, 5, 7057, 161, 19, 4912, 16, 1626, 5981, 11, 186, 7540, 16, 1276, 15, 2296, 7, 5718, 2367, 14621, 4161, 57, 4125, 387, 5, 15059, 7, 30, 8, 4653, 4939, 711, 747, 522, 17879, 788, 12, 1783, 44, 8, 15763, 6029, 1813, 9, 7472, 5, 1404, 1623, 11, 5699, 277, 130, 4161, 57, 18368, 16, 20126, 16496, 227, 8, 2473, 5895, 15, 147, 89, 22411, 139, 8, 1511, 5, 1485, 3271, 3, 21926, 9, 472, 19623, 5251, 8, 616, 12, 15614, 8, 1783, 5, 37, 13818, 10564, 15, 26, 3, 9, 3, 19513, 1481, 6, 18368, 186, 1328, 2605, 30, 7488, 1887, 3, 18, 8, 711, 2309, 9517, 89, 355, 5, 3966, 1954, 9233, 15, 6, 113, 293, 7, 8, 16548, 13363, 106, 14022, 84, 47, 14621, 4161, 6, 243, 255, 228, 59, 7828, 8, 1249, 18, 545, 11298, 1773, 728, 8, 8347, 1560, 5, 611, 6, 255, 243, 72, 1709, 1528, 161, 228, 43, 118, 4006, 91, 12, 766, 8, 3, 19513, 1481, 410, 59, 5124, 5, 96, 196, 17, 19, 1256, 68, 27, 103, 317, 132

In [41]:
# train, val and test splits will all be tokenized
# set load from cache to False if you want to re-run
tokenized_dataset = raw_dataset.map(preprocess_function, batched=True, load_from_cache_file=True)

  0%|          | 0/205 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [42]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
# load in a pre-trained model 
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [44]:
# set training arguments 
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-xsum-finetuning-test",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True, # to properly generate summaries
    fp16=True,
    push_to_hub=True,
)

In [48]:
# helps to pad inputs and labels to max len in the batch
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [49]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [52]:
# pass in pre-trained model, traning parameters (args) etc. 
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.8/dist-packages/huggingface_hub/repository.py:725: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  warnings.warn(
Cloning https://huggingface.co/pravsels/t5-small-xsum-finetuning-test into local empty directory.
Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: summary, id, document. If summary, id, document are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 204045
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 12753
  Number of trainable parameters = 60506624
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method 

Epoch,Training Loss,Validation Loss


Saving model checkpoint to t5-small-xsum-finetuning-test/checkpoint-500
Configuration saved in t5-small-xsum-finetuning-test/checkpoint-500/config.json
Model weights saved in t5-small-xsum-finetuning-test/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-small-xsum-finetuning-test/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-small-xsum-finetuning-test/checkpoint-500/special_tokens_map.json
tokenizer config file saved in t5-small-xsum-finetuning-test/tokenizer_config.json
Special tokens file saved in t5-small-xsum-finetuning-test/special_tokens_map.json
Saving model checkpoint to t5-small-xsum-finetuning-test/checkpoint-1000
Configuration saved in t5-small-xsum-finetuning-test/checkpoint-1000/config.json
Model weights saved in t5-small-xsum-finetuning-test/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-small-xsum-finetuning-test/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5-small-xsum-finetuning-test/c

In [ ]:
trainer.push_to_hub()

In [ ]:
# load in pushed model 
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("pravsels/t5-small-xsum-finetuning-test")